In [11]:
import os;
import json;
import builtins
import duckdb
import pandas as pd;
import matplotlib.pyplot as plt


def get_dataset_from_testcase(run):
    return run[0:-2]

def get_dataset_size(dataset):
    if dataset == "osm" or dataset == "books":
        return 800_000_000
    else:
        return 200_000_000

dir = '/home/chesetti/Repos/KVector_Merge/sponge/join_all'
runs = []
for test_case in os.listdir(dir):
    if test_case == 'build':
        continue
    for run_name in os.listdir(os.path.join(dir, test_case, 'outputs', 'results', 'run')):
        json_file = builtins.open(os.path.join(dir, test_case, 'outputs', 'results', 'run', run_name))
        run = (json.load(json_file))
        run['dataset'] = get_dataset_from_testcase(test_case)
        run['dataset_size'] = get_dataset_size(run['dataset'])
        runs.append(run)
df = pd.json_normalize(runs)
print(df.columns)

Index(['command', 'dataset', 'dataset_size', 'result.checksum',
       'result.duration_ns', 'result.duration_sec', 'result.inner_disk_fetch',
       'result.inner_disk_fetch_size', 'result.inner_total_bytes_fetched',
       'result.num_output_keys', 'result.outer_disk_fetch',
       'result.outer_disk_fetch_size', 'result.outer_total_bytes_fetched',
       'spec.algo', 'spec.algo_name', 'spec.check_checksum', 'spec.common_key',
       'spec.index.epsilon', 'spec.index.leaf_size_in_pages',
       'spec.index.search', 'spec.index.type', 'spec.inner_table',
       'spec.key_size', 'spec.key_type', 'spec.load_sstable_in_mem',
       'spec.name', 'spec.num_threads', 'spec.outer_table', 'spec.result_path',
       'spec.value_size', 'spec.write_result_to_disk'],
      dtype='object')


In [12]:
def get_index_type(index):
    if "btree" in index:
        return "BTREE"
    if "pgm" in index:
        return "PGM"
    return "NA"

def get_index_variant(index):
    if index == "sort_join" or index == "hash_join":
        return "NA"
    pos = index.find("_")
    return index[pos+1:-1]

df["threads"] = df["spec.num_threads"]
df["epsilon"] = df["spec.index.epsilon"]
df["duration_sec"] = df["result.duration_ns"] / (1000000000)
df["ratio"] = df["spec.common_key"]
df["thput"] = (df["result.num_output_keys"] / df["ratio"]) / (df["result.duration_ns"] / (1000000000))
df["algo"] = df["spec.algo_name"]
df["join_algo"] = df["spec.algo"]
df["index_type"] = df["spec.algo_name"].map(lambda x: get_index_type(x))
df["index_variant"] = df["spec.algo_name"].map(lambda x: get_index_variant(x))
df["inner_bytes_fetched"] = df["result.inner_total_bytes_fetched"]
display(df["algo"].unique())
display(df["dataset"].unique())
display(df["index_type"].unique())


array(['inlj_btree1024', 'lsj_pgm4096', 'lsj_sampledflatpgm4096',
       'hash_join', 'lsj_sampledflatpgm1024', 'inlj_flatpgm256',
       'lsj_btree4096', 'sort_join', 'inlj_sampledflatpgm4096',
       'inlj_btree4096', 'lsj_flatpgm4096', 'lsj_btree1024',
       'inlj_pgm256', 'lsj_btree256', 'lsj_flatpgm1024', 'inlj_pgm4096',
       'inlj_sampledflatpgm1024', 'lsj_pgm256', 'inlj_sampledflatpgm256',
       'lsj_pgm1024', 'lsj_sampledflatpgm256', 'inlj_flatpgm4096',
       'lsj_flatpgm256', 'inlj_flatpgm1024', 'inlj_pgm1024',
       'inlj_btree256'], dtype=object)

array(['wiki', 'uniform_dense', 'normal', 'uniform_sparse', 'books',
       'osm', 'lognormal', 'fb'], dtype=object)

array(['BTREE', 'PGM', 'NA'], dtype=object)

In [13]:
# SingleThread, HJ vs SJ vs INLJ(BTree256) vs INLJ(PGM256) vs LS(BTree256) vs LS(PGM256)
def plot_dataset_join_duration(dataset):
    rows = duckdb.sql(
        "SELECT ratio, threads, algo, MEDIAN(duration_sec) as d, MEDIAN(thput) as t FROM df " 
        "WHERE" 
        "   (algo='hash_join' OR algo='sort_join' OR algo='inlj_btree256' OR algo='inlj_pgm256' OR "
        "   algo='lsj_btree256' OR algo='lsj_flatpgm256')" 
        "   AND threads=1"
        f"   AND dataset='{dataset}'"
        "GROUP BY dataset, ratio, threads, algo"
    ).df()
    return(rows.pivot(index='ratio', values='d', columns=['algo']))

real_datasets = ['fb', 'wiki', 'osm', 'books']
synth_datasets = ['uniform_dense', 'uniform_sparse', 'normal', 'lognormal']

for dataset in real_datasets:
    display(dataset)
    display(plot_dataset_join_duration(dataset))

for dataset in synth_datasets:
    display(dataset)
    display(plot_dataset_join_duration(dataset))

'fb'

algo,hash_join,inlj_btree256,inlj_pgm256,lsj_btree256,lsj_flatpgm256,sort_join
ratio,,,,,,
1,79.785572,53.723555,33.464064,52.649181,26.987002,25.162671
10,53.982077,5.690409,4.085079,5.636519,3.016305,7.079598
100,44.572595,1.196704,0.943092,1.200733,0.740194,5.148352
1000,20.136223,0.225369,0.225207,0.227195,0.218850,4.931778


'wiki'

algo,hash_join,inlj_btree256,inlj_pgm256,lsj_btree256,lsj_flatpgm256,sort_join
ratio,,,,,,
1,32.578296,23.270138,14.453964,22.860397,12.128019,11.324560
10,24.205699,4.822314,3.162925,4.746160,2.529570,4.194891
100,20.306184,0.869956,0.698924,0.867694,0.503725,2.513819
1000,8.994534,0.208226,0.180925,0.208636,0.173009,2.319318


'osm'

...

algo,hash_join,inlj_btree256,inlj_pgm256,lsj_btree256,lsj_flatpgm256,sort_join
ratio,,,,,,
1,79.824059,52.790627,31.543096,51.743549,26.768670,24.984327
10,53.865636,5.705050,3.956811,5.632568,3.010531,7.082598
100,44.547378,1.230551,0.937140,1.231746,0.756002,5.154212
1000,22.689613,0.236518,0.231072,0.237742,0.226380,4.946246


'normal'

algo,hash_join,inlj_btree256,inlj_pgm256,lsj_btree256,lsj_flatpgm256,sort_join
ratio,,,,,,
1,80.190714,53.444584,31.871422,52.369877,26.820202,25.023829
10,53.703619,5.877991,4.001172,5.775892,3.066949,7.252469
100,44.970908,1.265768,0.947298,1.259856,0.766837,5.296070
1000,22.897787,0.240616,0.234836,0.240155,0.230603,5.102566


'lognormal'

algo,hash_join,inlj_btree256,inlj_pgm256,lsj_btree256,lsj_flatpgm256,sort_join
ratio,,,,,,
1,78.720935,52.932648,33.314757,51.819811,25.668690,24.880721
10,53.873789,5.701944,4.224656,5.654821,2.961539,7.065209
100,44.420552,1.213984,1.024411,1.214580,0.755119,5.132633
1000,19.661215,0.231155,0.237930,0.230680,0.228448,4.928607


In [14]:
# Effect of Epsilon on INLJ (BTree and PGM)
def plot_epsilon_inlj(dataset):
    rows = duckdb.sql(
        "SELECT ratio, algo, join_algo, epsilon, MEDIAN(duration_sec) as d, MEDIAN(thput) as t FROM df " 
        "WHERE" 
        "   join_algo = 'inlj'"
        "   AND threads=1"
        f"   AND dataset='{dataset}'"
        "GROUP BY dataset, ratio, algo, join_algo, epsilon"
    ).df()
    return rows.pivot(index='ratio', values='d', columns=['algo', 'epsilon'])

for dataset in real_datasets:
    display(dataset)
    display(plot_epsilon_inlj(dataset))

for dataset in synth_datasets:
    display(dataset)
    display(plot_epsilon_inlj(dataset))

'fb'

algo,inlj_flatpgm256,inlj_flatpgm1024,inlj_btree1024,inlj_btree4096,inlj_sampledflatpgm256,inlj_pgm1024,inlj_flatpgm4096,inlj_sampledflatpgm4096,inlj_btree256,inlj_pgm256,inlj_pgm4096,inlj_sampledflatpgm1024
epsilon,256.0,1024.0,1024.0,4096.0,256.0,1024.0,4096.0,4096.0,256.0,256.0,4096.0,1024.0
ratio,,,,,,,,,,,,
1,30.652944,32.459946,54.588636,52.968209,38.252170,33.376552,33.575393,34.713387,53.723555,33.464064,36.123307,35.019604
10,3.843559,3.819236,5.497925,5.307913,5.138802,3.915549,3.931653,4.218466,5.690409,4.085079,4.183998,4.537026
100,0.921725,0.949179,0.909161,0.809161,1.276552,0.970635,0.995085,1.086608,1.196704,0.943092,1.022775,1.176709
1000,0.223829,0.274128,0.286764,0.307143,0.252409,0.277599,0.384095,0.401380,0.225369,0.225207,0.384734,0.300688


'wiki'

algo,inlj_pgm256,inlj_sampledflatpgm1024,inlj_sampledflatpgm256,inlj_btree256,inlj_sampledflatpgm4096,inlj_flatpgm256,inlj_flatpgm4096,inlj_btree4096,inlj_pgm1024,inlj_flatpgm1024,inlj_btree1024,inlj_pgm4096
epsilon,256.0,1024.0,256.0,256.0,4096.0,256.0,4096.0,4096.0,1024.0,1024.0,1024.0,4096.0
ratio,,,,,,,,,,,,
1,14.453964,15.109639,15.390420,23.270138,15.455615,13.696504,14.929495,21.523201,14.999373,14.482200,22.631037,15.827330
10,3.162925,3.531694,3.716301,4.822314,3.422134,3.041385,3.234768,4.258731,3.296488,3.120329,4.524638,3.424825
100,0.698924,0.913820,0.978394,0.869956,0.814383,0.655918,0.716621,0.615451,0.713232,0.706915,0.689098,0.736530
1000,0.180925,0.246377,0.216119,0.208226,0.302543,0.181253,0.274723,0.181401,0.224906,0.216769,0.207184,0.278255


'osm'

...

algo,inlj_sampledflatpgm1024,inlj_flatpgm256,inlj_pgm256,inlj_pgm4096,inlj_flatpgm4096,inlj_btree256,inlj_pgm1024,inlj_sampledflatpgm4096,inlj_btree4096,inlj_flatpgm1024,inlj_btree1024,inlj_sampledflatpgm256
epsilon,1024.0,256.0,256.0,4096.0,4096.0,256.0,1024.0,4096.0,4096.0,1024.0,1024.0,256.0
ratio,,,,,,,,,,,,
1,31.503087,29.983271,31.543096,32.611604,31.841783,52.790627,31.160718,32.114671,52.300837,31.027543,53.698649,33.020995
10,4.152439,3.776262,3.956811,3.856450,3.804095,5.705050,3.766312,3.894787,5.306251,3.747043,5.494754,4.967536
100,1.189227,0.917394,0.937140,0.992095,0.990347,1.230551,0.984568,1.081915,0.826849,0.980722,0.924741,1.310415
1000,0.312569,0.228666,0.231072,0.397535,0.400153,0.236518,0.289575,0.421276,0.323396,0.290428,0.301985,0.256023


'normal'

algo,inlj_flatpgm256,inlj_btree4096,inlj_sampledflatpgm1024,inlj_flatpgm1024,inlj_btree256,inlj_btree1024,inlj_pgm4096,inlj_flatpgm4096,inlj_sampledflatpgm256,inlj_sampledflatpgm4096,inlj_pgm256,inlj_pgm1024
epsilon,256.0,4096.0,1024.0,1024.0,256.0,1024.0,4096.0,4096.0,256.0,4096.0,256.0,1024.0
ratio,,,,,,,,,,,,
1,29.944082,52.923079,32.114858,31.539024,53.444584,54.541270,35.093939,33.292856,31.614064,33.668926,31.871422,33.541749
10,3.806000,5.431510,4.645596,3.927536,5.877991,5.540704,4.475421,4.308542,4.970013,5.003237,4.001172,4.135038
100,0.925729,0.835695,1.318857,1.019723,1.265768,0.937591,1.222520,1.214645,1.329774,1.478316,0.947298,1.045134
1000,0.231377,0.329501,0.323160,0.298296,0.240616,0.304140,0.431134,0.424541,0.263512,0.455504,0.234836,0.299738


'lognormal'

algo,inlj_btree1024,inlj_sampledflatpgm256,inlj_pgm1024,inlj_flatpgm256,inlj_btree256,inlj_sampledflatpgm1024,inlj_pgm4096,inlj_pgm256,inlj_flatpgm4096,inlj_btree4096,inlj_flatpgm1024,inlj_sampledflatpgm4096
epsilon,1024.0,256.0,1024.0,256.0,256.0,1024.0,4096.0,256.0,4096.0,4096.0,1024.0,4096.0
ratio,,,,,,,,,,,,
1,53.684522,31.834472,33.354719,30.133240,52.932648,31.939601,35.825353,33.314757,33.705478,52.155415,31.595328,34.064562
10,5.505030,5.153610,4.261129,3.900504,5.701944,4.967971,4.790636,4.224656,4.624012,5.347522,4.068290,5.560521
100,0.911732,1.421147,1.158466,0.979978,1.213984,1.488593,1.469182,1.024411,1.437676,0.812216,1.140116,1.786009
1000,0.303248,0.261101,0.306920,0.231444,0.231155,0.332593,0.504190,0.237930,0.470993,0.314207,0.303786,0.501250


In [15]:
# Effect of Epsilon on INLJ (BTree and PGM)
def plot_epsilon_lsj(dataset):
    rows = duckdb.sql(
        "SELECT ratio, algo, join_algo, epsilon, MEDIAN(duration_sec) as d, MEDIAN(thput) as t FROM df " 
        "WHERE" 
        "   join_algo = 'lsj'"
        "   AND threads=1"
        f"   AND dataset='{dataset}'"
        "GROUP BY dataset, ratio, algo, join_algo, epsilon"
    ).df()
    return rows.pivot(index='ratio', values='d', columns=['algo', 'epsilon'])

for dataset in real_datasets:
    display(dataset)
    display(plot_epsilon_lsj(dataset))

for dataset in synth_datasets:
    display(dataset)
    display(plot_epsilon_lsj(dataset))

'fb'

algo,lsj_btree1024,lsj_flatpgm1024,lsj_sampledflatpgm4096,lsj_pgm1024,lsj_pgm4096,lsj_btree4096,lsj_pgm256,lsj_sampledflatpgm256,lsj_flatpgm4096,lsj_flatpgm256,lsj_sampledflatpgm1024,lsj_btree256
epsilon,1024.0,1024.0,4096.0,1024.0,4096.0,4096.0,256.0,256.0,4096.0,256.0,1024.0,256.0
ratio,,,,,,,,,,,,
1,52.765188,28.474029,30.610882,31.524737,35.218450,51.813857,31.933872,27.985456,29.859236,26.987002,28.824079,52.649181
10,5.427577,3.010683,3.183010,3.388625,3.654325,5.227907,3.532868,3.105649,3.153408,3.016305,3.054856,5.636519
100,0.901017,0.613775,0.574284,0.652138,0.626493,0.811296,0.798612,0.770686,0.610235,0.740194,0.621495,1.200733
1000,0.288444,0.253435,0.273259,0.260004,0.277396,0.315796,0.226063,0.242841,0.275550,0.218850,0.268029,0.227195


'wiki'

algo,lsj_flatpgm4096,lsj_sampledflatpgm256,lsj_flatpgm256,lsj_pgm256,lsj_pgm1024,lsj_sampledflatpgm4096,lsj_sampledflatpgm1024,lsj_pgm4096,lsj_btree256,lsj_btree4096,lsj_btree1024,lsj_flatpgm1024
epsilon,4096.0,256.0,256.0,256.0,1024.0,4096.0,1024.0,4096.0,256.0,4096.0,1024.0,1024.0
ratio,,,,,,,,,,,,
1,13.654480,12.719507,12.128019,13.852011,14.158023,13.754466,12.800034,15.187852,22.860397,21.121723,22.511216,12.547229
10,2.726041,2.579863,2.529570,2.818926,2.864973,2.755398,2.615648,3.071957,4.746160,4.226802,4.485158,2.600654
100,0.444253,0.523436,0.503725,0.538358,0.488916,0.446688,0.460861,0.479688,0.867694,0.612697,0.676807,0.451815
1000,0.160087,0.195386,0.173009,0.176829,0.176845,0.168344,0.178203,0.163821,0.208636,0.181598,0.206332,0.173633


'osm'

...

algo,lsj_flatpgm4096,lsj_btree256,lsj_flatpgm256,lsj_sampledflatpgm1024,lsj_btree4096,lsj_flatpgm1024,lsj_sampledflatpgm256,lsj_pgm256,lsj_btree1024,lsj_pgm4096,lsj_sampledflatpgm4096,lsj_pgm1024
epsilon,4096.0,256.0,256.0,1024.0,4096.0,1024.0,256.0,256.0,1024.0,4096.0,4096.0,1024.0
ratio,,,,,,,,,,,,
1,29.883543,51.743549,26.768670,28.602864,51.374448,28.348640,27.468063,30.333008,52.749507,31.689784,30.165467,30.189751
10,3.164637,5.632568,3.010531,3.101642,5.252183,3.043242,3.103629,3.379430,5.417373,3.323338,3.234927,3.223437
100,0.592688,1.231746,0.756002,0.648281,0.816544,0.639914,0.788451,0.795113,0.914489,0.611171,0.613100,0.651727
1000,0.294354,0.237742,0.226380,0.286976,0.322849,0.271000,0.254252,0.231511,0.300796,0.290296,0.300379,0.270081


'normal'

algo,lsj_pgm4096,lsj_flatpgm4096,lsj_sampledflatpgm1024,lsj_pgm256,lsj_flatpgm256,lsj_btree256,lsj_btree4096,lsj_flatpgm1024,lsj_btree1024,lsj_pgm1024,lsj_sampledflatpgm256,lsj_sampledflatpgm4096
epsilon,4096.0,4096.0,1024.0,256.0,256.0,256.0,4096.0,1024.0,1024.0,1024.0,256.0,4096.0
ratio,,,,,,,,,,,,
1,33.094355,29.770474,28.875777,30.336636,26.820202,52.369877,52.070128,28.198332,52.972112,31.409620,27.938565,29.927734
10,3.510877,3.127781,3.147763,3.415590,3.066949,5.775892,5.288045,3.026056,5.472135,3.353272,3.168278,3.189761
100,0.662887,0.601905,0.643274,0.793542,0.766837,1.259856,0.859088,0.651120,0.917467,0.686029,0.809134,0.609589
1000,0.296629,0.289028,0.290480,0.232950,0.230603,0.240155,0.329414,0.271792,0.303025,0.277214,0.257686,0.293955


'lognormal'

algo,lsj_pgm4096,lsj_btree256,lsj_flatpgm4096,lsj_sampledflatpgm4096,lsj_btree4096,lsj_sampledflatpgm1024,lsj_sampledflatpgm256,lsj_flatpgm256,lsj_btree1024,lsj_flatpgm1024,lsj_pgm1024,lsj_pgm256
epsilon,4096.0,256.0,4096.0,4096.0,4096.0,1024.0,256.0,256.0,1024.0,1024.0,1024.0,256.0
ratio,,,,,,,,,,,,
1,32.166440,51.819811,28.495432,29.427815,51.340584,28.170980,27.486908,25.668690,52.666640,27.017295,30.556263,30.897326
10,3.398305,5.654821,3.053803,3.124540,5.242098,3.066378,3.102597,2.961539,5.396317,2.959799,3.316870,3.471716
100,0.614113,1.214580,0.579760,0.585797,0.803116,0.638884,0.789848,0.755119,0.905219,0.623597,0.657125,0.814019
1000,0.289814,0.230680,0.284259,0.284786,0.311328,0.287632,0.257202,0.228448,0.295275,0.274921,0.278461,0.233523
